이전 과정을 통해 Lasso,Ridge,ElasitcNet의 적용을 배웠지만,          
모델 향상을 일으키기엔 부족함을 알았다.
이는 파라미터의 최적해를 알 수 없는 현시점의 한계때문이라 판단.
최적해를 찾는 방법에 대해 알아보겠다

## 최적해

위의 과정을 이해하기 위해선 CV에 대한 이해가 필요하다

물론 우리는 이전 Lasso,Ridge,Elasitic 파라미터 최적화를 위한 CV이므로 아래의 방법들을 알면된다.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sklearn.linear_model as sklin
import sklearn.model_selection as skmod
import numpy as np
np.random.seed(10)

D=pd.read_csv("C:/Users/User/Desktop/kc_house_data.csv")
data=pd.DataFrame(D)
target=["price"]
features=data.columns.difference(["price",'id','date'])
category=["condition",'waterfront']
numeric=[x for x in features if x not in category]

train,test=skmod.train_test_split(data[target+numeric],train_size=0.7,random_state=10)

CV를 통한 파라미터 최적화는 `sklearn`을 이용하는 것이 더욱 편하다

In [8]:
Lasso_CV=sklin.LassoCV(alphas=[0.1,1,10],cv=5)
Ridge_CV=sklin.RidgeCV(alphas=[0.1,1,10],cv=5)
Elastic_CV=sklin.ElasticNetCV(alphas=[0.1,1,10],l1_ratio=[1,0,0.5,0.3,0.7],cv=5)

### CV를 통한 파라미터 최적해 찾기

In [9]:
for cv in [Lasso_CV,Ridge_CV,Elastic_CV]:
    cv.fit(X=train[numeric],y=train[target])

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\sit

결과확인

In [10]:
for cv in [Lasso_CV,Ridge_CV,Elastic_CV]:
    print(cv.alpha_)

10.0
0.1
10.0


In [11]:
Elastic_CV.l1_ratio_

1.0

위의 결과를 보아 우리는 Lasso 규제가 해당 모델에서 가장 좋은 모델임을 알수있다.

* 참고
위의 평가지표는 손실함수다.         
이는 `scoring`파라미터를 통해 바꿀 수 있으며 아래와 같은 파라미터들이 존재한다.

|함수|파라미터|
|--|--|
|MSE|'neg_mean_squared_error'|
|MAE|'neg_mean_absolute_error'|
|Rsquard|'r2'|

### 최적의 모델 추출

In [14]:
best_model=sklin.Lasso(alpha=10.0)

In [16]:
best_model.fit(X=train[numeric],y=train[target])

Lasso(alpha=10.0)

In [19]:
best_model.coef_

array([ 4.01250843e+04, -3.59215041e+04,  2.29152574e+02,  9.98163629e+04,
        5.77790289e+05, -2.30215795e+05,  1.73207932e+02,  1.36775920e+02,
        3.06742856e+00,  2.31076744e+01,  4.62195764e-02, -2.84617686e-01,
        7.72635764e+04, -2.74983281e+03,  1.58862865e+01, -6.10548620e+02])

In [20]:
best_model.intercept_

array([9077799.99026849])

회귀계수 확인 

In [27]:
coef=dict(zip(numeric,best_model.coef_))
coef.update({'intercept':best_model.intercept_})
pd.DataFrame(coef).T.rename(columns={0:'beta'})

,beta
bathrooms,4.012508e+04
bedrooms,-3.592150e+04
floors,2.291526e+02
grade,9.981636e+04
lat,5.777903e+05
long,-2.302158e+05
sqft_above,1.732079e+02
sqft_basement,1.367759e+02
sqft_living,3.067429e+00
sqft_living15,2.310767e+01


### 모델평가

In [30]:
import sklearn.metrics as skmet

In [31]:
skmet.r2_score(train[target],best_model.predict(train[numeric]))

0.677837752092378

안타깝게도 모델 개선이 안됐다.          
$\alpha$ 의 값은 보통 지수(예 0.1,1,10)으로 중가시키는 것이 일방적이라고 알려져있다